In [1]:
# Start writing code here...
import numpy as np 
import pandas as pd
import collections
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [48]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
rows = pd.read_csv("train.csv")
rows_train = rows[:50000]
rows_val = rows[50001:55000]

In [4]:
rows_train.to_csv('train_dset.csv', index=False)
rows_val.to_csv('val_dset.csv', index=False)

## Dataset

In [5]:
class YelpPolarity(Dataset):
    def __init__(self, csv_path):
        self.data_frame = pd.read_csv(csv_path)
    
    def __len__(self):
        print(len(self.data_frame))
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        data_instance = self.data_frame.iloc[idx]
        inp_string = data_instance[1]
        inp_string = inp_string[:min(char_len, len(inp_string))]
        if "A" not in alphabet: inp_string = inp_string.lower()
            
        X_lis = [len(alphabet)] * char_len
        X_lis[:len(inp_string)] = [char_map.get(char, len(alphabet)) for char in inp_string]
        
        X = torch.LongTensor(X_lis)
        y = torch.LongTensor([data_instance[0]-1])
        return X, y

In [73]:
class BagOfWords(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = torch.LongTensor(self.X[idx])
        y = torch.LongTensor([self.y[idx]])
        return X, y

In [6]:
from sklearn.model_selection import train_test_split
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&*˜‘+-=<>()[]{}"
char_map = {char:idx for idx, char in enumerate(alphabet)}
def generate_embedding_matrix(size=len(alphabet)):
    return torch.vstack([torch.eye(size), torch.zeros(size)])
batch_size = 128
char_len = 1014
dset_train = YelpPolarity('train.csv')
dset_val = YelpPolarity('val_dset.csv')
train_loader = torch.utils.data.DataLoader(dset_train, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(dset_val, batch_size=batch_size, shuffle=True, num_workers=4)

559999
559999
4999
4999


In [18]:
df = pd.read_csv('train.csv')
X_sentence = []
y = []
for i in range(0, 55000):
    data_instance = df.iloc[i]
    X_sentence.append(data_instance[1])
    y.append(data_instance[0] - 1)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_sentence)
X = X.toarray()

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [74]:
dset_train = BagOfWords(X_train, y_train)
dset_val = BagOfWords(X_val, y_val)
train_loader = torch.utils.data.DataLoader(dset_train, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(dset_val, batch_size=batch_size, shuffle=True, num_workers=4)

loader : data loaded training samples \
in_feats: input features \
val_data_loader: validation data loader


## Bag of words

In [27]:
class Model(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(Model, self).__init__()
        self.dropout = nn.Dropout(0.1)
        self.linear1 = nn.Linear(in_feats, 16)
        self.linear2 = nn.Linear(32, 16)
        self.linear3 = nn.Linear(16, 2)
        self.l1 = nn.Linear(in_feats, out_feats)
    def forward(self, X):
        m = nn.Sigmoid()
        ans = self.linear1(X)
        ans = self.linear3(ans)
        return ans

In [93]:
def run(model, loader, optimizer, criterion, run_type="train"):
    if run_type == "train":
        model.train()
    else:
        model.eval()
    outs = []
    lbl = []
    model.train()
    running_loss = 0
    total_iters = len(loader)
    iter = 0
    if run_type == "train":
        for (corpus, labels) in tqdm(loader):
            corpus = corpus.type(torch.FloatTensor)
            corpus = corpus.to(device)
            #labels = labels.type(torch.FloatTensor)
            labels = labels.to(device)
            outputs = model(corpus)
            labels = labels.squeeze()
            args, preds = torch.max(outputs, dim=1) 
            loss = F.cross_entropy(outputs, labels)
            running_loss += loss
            iter += 1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            args, ans = torch.max(outputs, dim=1)        
            outs.extend(ans.cpu().numpy())
            lbl.extend(labels.cpu().detach().numpy().tolist())
        mean_loss = running_loss/iter
        return outs, lbl, mean_loss
    else:
        for (corpus, labels) in loader:
            corpus = corpus.type(torch.FloatTensor)
            corpus = corpus.to(device)
            #labels = labels.type(torch.FloatTensor)
            labels = labels.to(device)
            outputs = model(corpus)
            labels = labels.squeeze()
            args, preds = torch.max(outputs, dim=1) 
            print(outputs)
            loss = F.cross_entropy(outputs, labels)
            running_loss += loss
            iter += 1
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            args, ans = torch.max(outputs, dim=1)        
            outs.extend(ans.cpu().numpy())
            lbl.extend(labels.cpu().detach().numpy().tolist())
        mean_loss = running_loss/iter
        return outs, lbl, mean_loss  

In [85]:
def output_metrics(val_loader):
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    outs,lbl,mean_loss = run(model, val_loader, optimizer, criterion, "eval")
    return sum(np.array(outs) == np.array(lbl))/len(lbl)
def output_match(outs, lbl):
    return (sum(np.array(outs) == np.array(lbl))/len(lbl))

In [89]:
#model = Model(1014).to(device)
model = Model(len(X_train[0]), 2).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
for i in range(0, 10):
    outs, lbl, mean_loss = run(model, train_loader, optimizer, criterion)
    print("Train Accuracy for iter, ",i, " : ", output_match(outs, lbl))
    print("Val Accuracy for iter, ",i, " : ", output_metrics(val_loader))
    print("Loss for iter, ",i, " : ", mean_loss.item())

  0%|          | 0/387 [00:00<?, ?it/s]

Train Accuracy for iter,  0  :  0.847030303030303
Val Accuracy for iter,  0  :  0.893090909090909
Loss for iter,  0  :  0.3771659731864929


  0%|          | 0/387 [00:00<?, ?it/s]

Train Accuracy for iter,  1  :  0.9007474747474747
Val Accuracy for iter,  1  :  0.9105454545454545
Loss for iter,  1  :  0.26623520255088806


  0%|          | 0/387 [00:00<?, ?it/s]

...

Train Accuracy for iter,  7  :  0.9324848484848485
Val Accuracy for iter,  7  :  0.9152727272727272
Loss for iter,  7  :  0.18369139730930328


  0%|          | 0/387 [00:00<?, ?it/s]

Train Accuracy for iter,  8  :  0.9334747474747475
Val Accuracy for iter,  8  :  0.914
Loss for iter,  8  :  0.18429367244243622


  0%|          | 0/387 [00:00<?, ?it/s]

Train Accuracy for iter,  9  :  0.9374343434343434
Val Accuracy for iter,  9  :  0.9149090909090909
Loss for iter,  9  :  0.1690542995929718


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f80c269c-dad1-4c63-bf97-6d9868e127ca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>